# Projet maison n° 3

In [1]:
# imports
import pandas as pd
import os
import re
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python `df_names_us()` qui produit un unique DataFrame avec tous les fichiers en utilisant **pandas** (par ex. avec la fonction `pandas.concat()`), pas de bash :)

Ordre et noms des colonnes du DataFrame : "year", "name", "gender", "births"

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini dans les différents fichiers (voir la documentation du dataset).

In [2]:
def df_names_us():
    df = pd.DataFrame()
    #list the files
    filelist = os.listdir( r'C:\Users\DELL\Desktop\names') 
    filelist.remove('NationalReadMe.pdf')
     
    #read
    df_list=[]
    for file in filelist:
        df_temp = pd.read_csv(r'C:/Users/DELL/Desktop/names/' +file, names = ['name', 'gender', 'births'] )
        df_temp['year'] = os.path.basename(file)[3:7]
        df_list.append(df_temp)
    #concatenate 
    df = pd.concat(df_list, ignore_index=True)
    return df[['year', 'name', 'gender', 'births']]

In [3]:
df_names_us()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746
...,...,...,...,...
2052776,2021,Zyeire,M,5
2052777,2021,Zyel,M,5
2052778,2021,Zyian,M,5
2052779,2021,Zylar,M,5


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2021_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction `df_names_fr()` qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : "year", "name", "gender", "births"
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne "gender"
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : year (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [4]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    df = pd.read_csv( r'C:\Users\DELL\Desktop\nat2021_csv\nat2021.csv', sep=';', keep_default_na=False)
    df.columns = ['gender','name','year','births']

    #drop unsuable data
    df.drop(df.loc[df['name'] == '_PRENOMS_RARES'].index, inplace=True)
    df.drop(df.loc[df['name'].str.len() < 2].index, inplace=True)
    df.drop(df.loc[df['year'] == 'XXXX'].index, inplace=True)
    df = df.dropna()

    #Modifications
    df['gender'] = df['gender'].map({1: 'M', 2: 'F'})
    df['name'] = df['name'].str.title()
    df = df.reindex(columns = ['year','name','gender','births'])

    #sort 
    df.sort_values(by=['year', 'gender', 'births', 'name'],ascending=[True, True, False, True],inplace=True,ignore_index=True)


    return df

In [5]:
df_names_fr()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696
...,...,...,...,...
648325,2021,Élyam,M,3
648326,2021,Énael,M,3
648327,2021,Éros,M,3
648328,2021,Étan,M,3


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

Les données sont dans le fichier "Webstat_Export.csv".

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3. On retiendra uniquement les colonnes se terminant effectivement par un code ISO3 entre parenthèses. Par ex., "Dollar des Etats-Unis (USD)".

Implémenter une fonction `df_taux_change()` qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction `pd.to_datetime()` avec le format`"%d/%m/%Y"`) et doit être trié par ordre croissant. Les colonnes du DataFrame doivent correspondre aux devises sélectionnées.

In [6]:
df = pd.read_csv( r'C:\Users\DELL\Desktop\Webstat_Export.csv', sep=';')
df.head(10)

,Titre :,Dollar australien (AUD),Lev bulgare (BGN),Real brésilien (BRL),Dollar canadien (CAD),Franc suisse (CHF),Yuan renminbi chinois (CNY),Livre chypriote (CYP),Couronne tchèque (CZK),Couronne danoise (DKK),...,Leu roumain (RON),Rouble russe (RUB),Couronne suédoise (SEK),Dollar de Singapour (SGD),Tolar slovène (SIT),Couronne slovaque (SKK),Baht thaïlandais (THB),Livre turque (TRY),Dollar des Etats-Unis (USD),Rand sud-africain (ZAR)
0,Code série :,EXR.D.AUD.EUR.SP00.A,EXR.D.BGN.EUR.SP00.A,EXR.D.BRL.EUR.SP00.A,EXR.D.CAD.EUR.SP00.A,EXR.D.CHF.EUR.SP00.A,EXR.D.CNY.EUR.SP00.A,EXR.D.CYP.EUR.SP00.A,EXR.D.CZK.EUR.SP00.A,EXR.D.DKK.EUR.SP00.A,...,EXR.D.RON.EUR.SP00.A,EXR.D.RUB.EUR.SP00.A,EXR.D.SEK.EUR.SP00.A,EXR.D.SGD.EUR.SP00.A,EXR.D.SIT.EUR.SP00.A,EXR.D.SKK.EUR.SP00.A,EXR.D.THB.EUR.SP00.A,EXR.D.TRY.EUR.SP00.A,EXR.D.USD.EUR.SP00.A,EXR.D.ZAR.EUR.SP00.A
1,Unité :,Dollar Australien (AUD),Lev Nouveau (BGN),Real Bresilien (BRL),Dollar Canadien (CAD),Franc Suisse (CHF),Yuan Ren Min Bi (CNY),Livre Cypriote (CYP),Couronne Tcheque (CZK),Couronne Danoise (DKK),...,Nouveau Ron (RON),Rouble Russe (RUB) (RUB),Couronne Suedoise (SEK),Dollar de Singapour (SGD),Tolar (SIT),Couronne Slovaque (SKK),Baht (THB),Nouvelle Livre Turque (TRY),Dollar des Etats-Unis (USD),Rand (ZAR)
2,Magnitude :,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),...,Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0),Unités (0)
3,Méthode d'observation :,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),...,Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A),Moyenne de la période (A)
4,Source :,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),...,BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0),BCE (Banque Centrale Européenne) (4F0)
5,20/09/2022,"1,4893","1,9558","5,2139","1,3268","0,9644","7,003",NaN,"24,556","7,4368",...,"4,934",NaN,"10,8338","1,4074",NaN,NaN,"36,968","18,2833","0,9986","17,7261"
6,19/09/2022,"1,495","1,9558","5,2886","1,3294","0,9658","7,0066",NaN,"24,494","7,4373",...,"4,93",NaN,"10,7993","1,4082",NaN,NaN,"36,983","18,2738","0,999","17,7267"
7,18/09/2022,-,-,-,-,-,-,NaN,-,-,...,-,NaN,-,-,NaN,NaN,-,-,-,-
8,17/09/2022,-,-,-,-,-,-,NaN,-,-,...,-,NaN,-,-,NaN,NaN,-,-,-,-
9,16/09/2022,"1,4894","1,9558","5,2279","1,3226","0,9579","6,9787",NaN,"24,497","7,4366",...,"4,9238",NaN,"10,7541","1,4025",NaN,NaN,"36,8","18,1923","0,9954","17,6004"


In [7]:
# taux de change
def df_taux_change(devises):
    data = pd.read_csv(r'C:\Users\DELL\Desktop\Webstat_Export.csv', sep = ';', skiprows = range(1,6))
    df = pd.DataFrame(data)
    
    iso = [re.search(r'\([A-Z]{3}\)', word) for word in df.columns]
    for i in range(len(iso)):
        if iso[i]!= None:
            iso[i] = iso[i].group(0).replace('(','').replace(')','')
    iso[0] = 'date'            
    df.columns=iso

    df['date'] = pd.to_datetime(df['date'], format="%d/%m/%Y")
    df.set_index('date', inplace=True)
    df = df[devises]
    
    df = df.apply(lambda x: x.str.replace(',','.'))
    df = df.apply(lambda x: x.replace('-',np.NaN) )
    df=df.astype('float')
    
    df.dropna(inplace=True)
    
    df.sort_index(inplace=True)
    
    
    return df

In [8]:
devises = ['GBP','USD']
df_taux_change(devises)

,GBP,USD
date,,
1999-01-04,0.71110,1.1789
1999-01-05,0.71220,1.1790
1999-01-06,0.70760,1.1743
1999-01-07,0.70585,1.1632
1999-01-08,0.70940,1.1659
...,...,...
2022-09-14,0.86498,0.9990
2022-09-15,0.86934,0.9992
2022-09-16,0.87400,0.9954


### Tests

In [9]:
import unittest

class Session3Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 2052781)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ["year", "name", "gender", "births"])
        # lignes
        self.assertEqual(len(df), 648330)
        # index
        self.assertTrue(isinstance(df.index, pd.RangeIndex))
        # test names
        self.assertTrue(df.loc[df.name.str.contains(r"^[A-Z]+$")].empty)
        self.assertTrue(df.loc[df.name.str.contains(r"-[a-z]+$")].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df.gender=="F"]) + len(df.loc[df.gender=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(["CHF", "GBP", "USD"])
        # colonnes
        self.assertEqual(list(df.columns), ["CHF", "GBP", "USD"])
        # index
        self.assertTrue(isinstance(df.index, pd.DatetimeIndex))
        # index trié
        self.assertEqual(list(df.index.argsort()), list(range(len(df))))
        # types taux
        self.assertTrue((df.dtypes == float).all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [10]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Session3Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [11]:
# run tests
run_tests()

test_df_names_fr (__main__.Session3Tests) ... ok
test_df_names_us (__main__.Session3Tests) ... ok
test_df_taux_change (__main__.Session3Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 7.068s

OK
